<a href="https://colab.research.google.com/github/Diljitsingh14/Colab-Notebooks/blob/main/Software_toolsAssignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Team Groot
## C0885081 Syed Zumair Hussain
## C0882038 Diljit Singh

In [ ]:
import re
import nltk
import pandas as pd
import numpy as np
import unicodedata
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Creating dataframe from csv file
df=pd.read_csv("/content/drive/MyDrive/TwitterHate.csv",index_col='id')

In [ ]:
df

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation
...,...,...
31958,0,ate @user isz that youuu?ðððððð...
31959,0,to see nina turner on the airwaves trying to...
31960,0,listening to sad songs on a monday morning otw...


### Here we do not need the Id column, so we are dropping it

## Data Preprocessing

### checking for null values

In [ ]:
df.isnull().sum()

label    0
tweet    0
dtype: int64

### No Missing values Found!!!

### Checking for any duplicate data

In [ ]:
df.duplicated().sum()

2432

In [ ]:
## Now dropping the duplicate
df.drop_duplicates(inplace=True)

### Duplicates removed!!!

In [ ]:
## Consider subset of the data due to computational issue
df=df.sample(frac=0.5)

## Applying Different NLP Techniques for Cleaning the tweet column

### Function to convert text to lowercase,remove (/r, /n carriage and newline characters),remove User Handles Ex: @user, @user123 .... ,URL,Numbers and punctuations, Also This symbol 'ð'

In [ ]:
def clean_function(string):
    string = string.replace('\r','').replace('\n',' ').lower()
    string = re.sub(r'@\w+\s?', '', string)
    string = re.sub(r'http[s]?://\S+|www\.\S+','',string)
    string = re.sub(r'\d+','',string)
    string = re.sub(r'[^\w\s]', '', string)
    string = re.sub(r'ð','',string)
    return string

### Function to normalize text by removing diacrities(accents) EX:- café => cafe

In [ ]:
def remove_diacritics(text):

    # Normalizing the text to decomposed form (NFD) so that we can handle combined characters.
    normalized_text = unicodedata.normalize('NFD', text)

    # Filtering out the characters whose combining class is non-zero (indicating diacritics).
    text_without_diacritics = ''.join(
        c for c in normalized_text if not unicodedata.combining(c)
    )

    # Re-compose to NFC
    composed_text = unicodedata.normalize('NFC', text_without_diacritics)

    return composed_text

### Function to do Spelling check and correct Using TextBlob

In [ ]:
def spell_correct(text):
    txtblob = TextBlob(text)
    corrected_text = txtblob.correct()
    return corrected_text

### Function to Tokenize the words

In [ ]:
def tokenize_words(string):
    tokenized_word = word_tokenize(string)
    return tokenized_word

### Function for removing the stop words

In [ ]:
def stopwords_func(tokenized_word):
    filtered_token = []
    stopword_list = stopwords.words('english')
    for i in tokenized_word:
        if i not in stopword_list:
            filtered_token.append(i)
    return filtered_token

### Function to apply Lemmitization

In [ ]:
def lemmitization(tokenized_string1):
    lemitization_list=[]
    lem=WordNetLemmatizer()
    lemmatized_words = [lem.lemmatize(word,pos='v') for word in tokenized_string1]

    return lemmatized_words

### After seeing the plots we see that there are some worsd with 1 and two letter which does't have any meaning or context so removing those letters

## Function to removing 1 and 2 letter words

In [ ]:
def remove_1_and_2(words_list):
    # Remove words with 1 or 2 characters
    filtered_words = [word for word in words_list if len(word) > 2]

    return filtered_words

## Function to get word index

In [ ]:
def create_word_index(tokenized_words):
    word_index = {word: index + 1 for index, word in enumerate(tokenized_words)}
    return word_index

In [ ]:
def preprocess_text(df):
    df['tweet'] = (
        df['tweet']
        .apply(clean_function)
        .apply(remove_diacritics)
        .apply(tokenize_words)
        .apply(stopwords_func)
        .apply(lemmatization)
        .apply(remove_1_and_2)
        .apply(lambda x: ' '.join(x))
    )

    return df


In [ ]:
PreprocessText(df)

# Join the 'clean_tweet' column into a single list (corpus)
corpus = df['tweet'].tolist()

# Print the result
for sentence in corpus:
    print(sentence)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Creating Tokenizer on corpus

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

sequences = tokenizer.texts_to_sequences(corpus)

# displat vocab
print("Vocabulary:", tokenizer.word_index)
print(f"number of tokens: {len(tokenizer.word_index)}")
print(f"sequence lenght: {len(sequences)}")

Vocabulary: {'love': 1, 'day': 2, 'get': 3, 'happy': 4, 'amp': 5, 'make': 6, 'like': 7, 'life': 8, 'new': 9, 'today': 10, 'time': 11, 'see': 12, 'father': 13, 'people': 14, 'bihday': 15, 'cant': 16, 'one': 17, 'come': 18, 'smile': 19, 'good': 20, 'dont': 21, 'feel': 22, 'look': 23, 'work': 24, 'want': 25, 'wait': 26, 'say': 27, 'thank': 28, 'weekend': 29, 'think': 30, 'summer': 31, 'need': 32, 'fun': 33, 'know': 34, 'days': 35, 'live': 36, 'positive': 37, 'friday': 38, 'great': 39, 'thankful': 40, 'beautiful': 41, 'family': 42, 'friends': 43, 'first': 44, 'world': 45, 'even': 46, 'orlando': 47, 'back': 48, 'week': 49, 'watch': 50, 'morning': 51, 'best': 52, 'take': 53, 'tomorrow': 54, 'way': 55, 'home': 56, 'music': 57, 'never': 58, 'happiness': 59, 'sad': 60, 'really': 61, 'fathersday': 62, 'find': 63, 'sunday': 64, 'cute': 65, 'blog': 66, 'right': 67, 'trump': 68, 'much': 69, 'night': 70, 'play': 71, 'pay': 72, 'always': 73, 'show': 74, 'still': 75, 'next': 76, 'girl': 77, 'last': 78

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 264

# Padding for same sent length
padded_sequences = pad_sequences(sequences,maxlen=max_sequence_length)

# Vocab length
vocab_size = len(tokenizer.word_index) + 1
vocab_size

24095

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, BatchNormalization

embedding_dim = 128

# NN model Architecture
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    LSTM(128),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(2, activation='sigmoid')  # Changed output dimension and activation function
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 264, 128)          3084160   
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense_4 (Dense)             (None, 32)                4128      
                                                                 
 batch_normalization_2 (Bat  (None, 32)                128       
 chNormalization)                                                
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                      

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(padded_sequences,to_categorical(df['label']) , test_size=0.2, random_state=42, shuffle=False)

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((11812, 264), (11812, 2), (2953, 264), (2953, 2))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=3)

hist = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=.2,callbacks=[early_stopping])


Epoch 1/15
296/296 [==============================] - 169s 559ms/step - loss: 0.3551 - accuracy: 0.8850 - val_loss: 0.2459 - val_accuracy: 0.9378
Epoch 2/15
296/296 [==============================] - 178s 601ms/step - loss: 0.1030 - accuracy: 0.9691 - val_loss: 0.2285 - val_accuracy: 0.9429
Epoch 3/15
296/296 [==============================] - 170s 573ms/step - loss: 0.0401 - accuracy: 0.9884 - val_loss: 0.2698 - val_accuracy: 0.9471
Epoch 4/15
296/296 [==============================] - 168s 568ms/step - loss: 0.0250 - accuracy: 0.9935 - val_loss: 0.2239 - val_accuracy: 0.9501
Epoch 5/15
117/296 [==========>...................] - ETA: 1:34 - loss: 0.0178 - accuracy: 0.9957

In [ ]:
# Plot training history
plt.plot(hist.history['accuracy'], label='Training Accuracy')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
plt.plot(hist.history['loss'], label='Training Loss')
plt.plot(hist.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
model.save('tweetClassifier.h5')